In [1]:
##Setup
import pandas as pd;
import os;
from urllib.parse import unquote

os.chdir('../..')

In [2]:
## Import all the coresignal files 
directory_path = 'data/1_raw_data/linkedin/coresignal/company/main'
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

cs_data = (pd.concat([pd.read_csv(os.path.join(directory_path, file), usecols=["id", "url"]).
          rename(columns={"id": "cs_id"})
          for file in csv_files], ignore_index=True)
          .assign(url = lambda df: df["url"].str.replace(r'^.*?(linkedin)', r'\1', regex=True).apply(unquote)))

## Import the Revelio files 
revelio_data = (pd.read_csv('data/1_raw_data/linkedin/revelio/company_mapping.csv',
                usecols=["rcid", "linkedin_url"],
                encoding='ISO-8859-1')
                .rename(columns={"linkedin_url": "url"})
               .assign(url = lambda df: df["url"].str.replace(r'^.*?(linkedin)', r'\1', regex=True)))

## merge the two
combined = pd.merge(cs_data, revelio_data, on="url", how= "outer", indicator=True)
combined_filtered = (combined[combined["url"]
                     .str.contains("linkedin",regex = False, na=False)]
                    .sort_values(by="url").reset_index(drop=True))
combined_filtered.to_csv('data/2_processed/linkedin/cs_revelio_crosswalk.csv', index=False)

In [6]:
matched_share = (combined_filtered[combined_filtered["cs_id"].notna()]["_merge"] == "both" ).mean()
matched_share

np.float64(0.17516368635330545)